<a href="https://colab.research.google.com/github/adityasahusomu/Youtube_Comment_Analyzer/blob/main/Exp_10_SVM_with_HP_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow boto3 awscli optuna lightgbm imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 142.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [2]:
import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [3]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': 'J5BY90817NZF1JXA', 'HostId': 'hXSqDq00jYmhAi598NnUlkHueBpqW8/j8C3TKoYlks8o4hjRkySvzCOpp8iU4d8Vp00Rd7TXAaE=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'hXSqDq00jYmhAi598NnUlkHueBpqW8/j8C3TKoYlks8o4hjRkySvzCOpp8iU4d8Vp00Rd7TXAaE=', 'x-amz-request-id': 'J5BY90817NZF1JXA', 'date': 'Fri, 17 Oct 2025 07:36:01 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'adityasahusomu02', 'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [6]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [7]:
mlflow.set_experiment("exp-10-SVM_with HP tuning")

<Experiment: artifact_location='s3://project1-mlflow-bucket/311215062922422246', creation_time=1760674042265, experiment_id='311215062922422246', last_update_time=1760674042265, lifecycle_stage='active', name='exp-10-SVM_with HP tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [9]:
url = "https://raw.githubusercontent.com/adityasahusomu/Youtube_Comment_Analyzer/refs/heads/main/cleaned_reddit_dataset.csv"
df = pd.read_csv(url).dropna(subset = ['clean_comment'])
df.shape

(36662, 2)

In [10]:

df = df.dropna(subset=['category'])
ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")


def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

run_optuna_experiment()


[I 2025-10-17 07:39:50,776] A new study created in memory with name: no-name-67c80f5f-ba08-486a-b7a9-421cfd4d7442
[I 2025-10-17 07:42:59,386] Trial 0 finished with value: 0.4482139082646375 and parameters: {'C': 0.0002598020916315596, 'kernel': 'linear'}. Best is trial 0 with value: 0.4482139082646375.
[I 2025-10-17 07:45:38,905] Trial 1 finished with value: 0.6946734305643627 and parameters: {'C': 0.03502904239473429, 'kernel': 'linear'}. Best is trial 1 with value: 0.6946734305643627.
[I 2025-10-17 07:48:53,495] Trial 2 finished with value: 0.3703233988585923 and parameters: {'C': 0.007543683719873065, 'kernel': 'poly'}. Best is trial 1 with value: 0.6946734305643627.
[I 2025-10-17 07:52:00,619] Trial 3 finished with value: 0.5541111815683788 and parameters: {'C': 0.027762271064391948, 'kernel': 'rbf'}. Best is trial 1 with value: 0.6946734305643627.
[I 2025-10-17 07:54:12,430] Trial 4 finished with value: 0.7447685478757133 and parameters: {'C': 0.15514976141518488, 'kernel': 'linea

🏃 View run SVM_SMOTE_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/311215062922422246/runs/def144e69d124341930d1a92f12809d0
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/311215062922422246
